In [ ]:
!pip install yfinance
!pip install pandas_datareader
!pip install matplotlib
!pip install psycopg2-binary python-dotenv

# prompt: pip3 install ta_lib

!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar xvzf ta-lib-0.4.0-src.tar.gz
import os
os.chdir('ta-lib')
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.9 MB/s eta 0:00:00
--2024-09-19 14:14:10--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2024-09-19 14:14:11--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://nchc.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz?viasf=1 [following]
--2024-09-19 14:14:11--  http://nchc.dl.sourceforge.ne

In [ ]:
from sqlalchemy import create_engine  # SQL database interaction
import pandas as pd  # Data manipulation and analysis
from pandas_datareader import data as pdr  # Data retrieval for financial data
import yfinance as yf  # Yahoo Finance API for financial data
from datetime import datetime, timedelta  # Date and time handling
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore",category=FutureWarning)
warnings.filterwarnings('ignore',category=Warning)
warnings.filterwarnings('ignore',category=DeprecationWarning)

import numpy as np
import time
import math
import os.path
import talib as ta

from tqdm.notebook import tqdm
from datetime import timedelta, datetime
from dateutil import parser
from sklearn.preprocessing import MinMaxScaler

import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from matplotlib.dates import DayLocator
from matplotlib.dates import MonthLocator
import matplotlib.ticker as mticker
import matplotlib.mlab as mlab

import seaborn as sns
sns.set()

plt.rcParams.update({'figure.figsize':(15,8), 'figure.dpi':120})


In [ ]:
# Make dataframe output easier to read
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Allow horizontal scrolling
pd.set_option('display.max_rows', None)  # Show all rows

# Import NASDAQ stock tickers that will loop through the yfinance library - to collect stock history data
url_nasdaq = "https://www.nasdaqtrader.com/dynamic/SymDir/nasdaqlisted.txt"
nasdaq_symbols_df = pd.read_csv(url_nasdaq, sep='|')
nasdaq_symbols = nasdaq_symbols_df['Symbol'].tolist()

nasdaq_symbols

['AACG',
 'AADI',
 'AADR',
 'AAGR',
 'AAGRW',
 'AAL',
 'AAME',
 'AAOI',
 'AAON',
 'AAPB',
 'AAPD',
 'AAPL',
 'AAPU',
 'AAXJ',
 'ABAT',
 'ABCL',
 'ABCS',
 'ABEO',
 'ABL',
 'ABLLL',
 'ABLLW',
 'ABLV',
 'ABLVW',
 'ABNB',
 'ABOS',
 'ABSI',
 'ABTS',
 'ABUS',
 'ABVC',
 'ABVE',
 'ABVEW',
 'ABVX',
 'ACAB',
 'ACABU',
 'ACABW',
 'ACAC',
 'ACACU',
 'ACACW',
 'ACAD',
 'ACB',
 'ACCD',
 'ACDC',
 'ACET',
 'ACGL',
 'ACGLN',
 'ACGLO',
 'ACHC',
 'ACHL',
 'ACHV',
 'ACIC',
 'ACIU',
 'ACIW',
 'ACLS',
 'ACLX',
 'ACMR',
 'ACNB',
 'ACNT',
 'ACON',
 'ACONW',
 'ACRS',
 'ACRV',
 'ACST',
 'ACT',
 'ACTG',
 'ACTU',
 'ACVA',
 'ACWI',
 'ACWX',
 'ACXP',
 'ADAG',
 'ADAP',
 'ADBE',
 'ADD',
 'ADEA',
 'ADGM',
 'ADI',
 'ADIL',
 'ADMA',
 'ADN',
 'ADNWW',
 'ADP',
 'ADPT',
 'ADSE',
 'ADSEW',
 'ADSK',
 'ADTN',
 'ADTX',
 'ADUS',
 'ADV',
 'ADVM',
 'ADVWW',
 'ADXN',
 'AEAE',
 'AEAEU',
 'AEAEW',
 'AEHL',
 'AEHR',
 'AEI',
 'AEIS',
 'AEMD',
 'AENT',
 'AENTW',
 'AEP',
 'AERT',
 'AERTW',
 'AEYE',
 'AFBI',
 'AFCG',
 'AFJK',
 'AFJKR',
 

In [ ]:
# Make dataframe output easier to read
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Allow horizontal scrolling
pd.set_option('display.max_rows', None)  # Show all rows

# Create dataframe containing GOOGL's price history per day for 2023 YTD
sample_data = yf.download('GOOGL', start='2023-01-01', end='2023-12-31')
sample_data.sort_values(by=['Date'], inplace=True, ascending=False)
sample_data.reset_index(inplace=True)
sample_data.head()
sample_data_df = pd.DataFrame(sample_data)

# Print sample data
sample_data_df

#OUTPUT_FOLDER = '/content/'
#sample_data_df.to_csv(OUTPUT_FOLDER + 'historical_data.csv', mode='a', index=False, header=True)


[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2023-12-29,139.630005,140.360001,138.779999,139.690002,139.344955,18727200
1,2023-12-28,140.779999,141.139999,139.750000,140.229996,139.883621,16045700
2,2023-12-27,141.589996,142.080002,139.889999,140.369995,140.023270,19628600
3,2023-12-26,141.589996,142.679993,141.190002,141.520004,141.170441,16780300
4,2023-12-22,140.770004,141.990005,140.710007,141.490005,141.140518,26514600
5,2023-12-21,139.490005,140.690002,139.179993,140.419998,140.073151,27488300
6,2023-12-20,138.970001,141.699997,138.070007,138.339996,137.998291,49107200
7,2023-12-19,136.839996,137.470001,136.080002,136.649994,136.312469,25476800
8,2023-12-18,132.630005,137.149994,132.429993,135.800003,135.464569,32258000
9,2023-12-15,131.619995,133.509995,131.179993,132.600006,132.272476,50815200


In [ ]:
startDate = pd.to_datetime('today') - 30 * pd.Timedelta(days=1)
endDate = pd.to_datetime('today')


print(startDate)
print(endDate)

2024-08-20 14:17:41.281368
2024-09-19 14:17:41.281905


In [ ]:
historical_data_df = pd.DataFrame()
historical_data_df.head()

""


In [ ]:
for ticker in nasdaq_symbols:
    try:
        # Attempt to download data for the last 5 years
        data = yf.download(ticker, start=startDate, end=endDate)

        # Drop and Add a new column for the ticker symbol
        data.drop(['Adj Close'], axis=1, inplace=True)

        data['Symbol'] = ticker
        data.sort_values(by=['Date'], inplace=True, ascending=False)
        data.reset_index(inplace=True)
        historical_data_df = pd.DataFrame(data)

        print(f"Downloaded data for {ticker}")
        OUTPUT_FOLDER = '/content/'
        historical_data_df.to_csv(OUTPUT_FOLDER + 'historical_data.csv', mode='a', index=False, header=False)
        historical_data_df.head()

    except Exception:
        # Suppress all error messages and continue with the next iteration
        continue

# Print the first few rows of the combined DataFrame


[*********************100%***********************]  1 of 1 completed


Downloaded data for AACG


[*********************100%***********************]  1 of 1 completed


Downloaded data for AADI


[*********************100%***********************]  1 of 1 completed


...

Downloaded data for ZYXI


[*********************100%***********************]  1 of 1 completed


Downloaded data for ZZZ


[*********************100%***********************]  3 of 4 completed
ERROR:yfinance:
4 Failed downloads:
ERROR:yfinance:['CREATION', '0919202410:01', 'TIME:']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
ERROR:yfinance:['FILE']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2024-08-20 14:17:41.281368 -> 2024-09-19 14:17:41.281905)')


Downloaded data for File Creation Time: 0919202410:01


In [ ]:
#crypto = pd.read_csv("/content/historical_data.csv", names=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Symbol'], parse_dates=True)
#crypto.head()

for ticker in nasdaq_symbols:
    try:
      crypto = pd.read_csv("/content/historical_data.csv", names=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Symbol'], parse_dates=True)
      head_aacg = crypto[crypto["Symbol"] == ticker]
      print(ticker)
      head_aacg = head_aacg[["Date","Open","High","Low","Close","Volume"]]
      print(head_aacg.head())
      head_aacg.head()
    except Exception:
      continue

head_aacg.head()

Streaming output truncated to the last 5000 lines.
74673  2024-09-19  11.34  11.72  11.34  11.3974    4474
74674  2024-09-18  11.95  11.99  11.74  11.7400    8800
74675  2024-09-17  11.72  11.93  11.72  11.9200   12200
74676  2024-09-16  11.49  11.73  11.45  11.6700    5900
74677  2024-09-13  11.38  11.48  11.38  11.3800    1100
SSKN
             Date    Open    High     Low  Close  Volume
74695  2024-09-19  3.1778  3.1778  2.9501   3.05    5750
74696  2024-09-18  3.0100  3.1700  2.9700   2.99   17200
74697  2024-09-17  3.0200  3.1900  3.0000   3.04    7600
74698  2024-09-16  3.0600  3.2000  2.9700   3.02   20300
74699  2024-09-13  3.1600  3.2000  2.9600   3.01    8300
SSNC
             Date       Open       High        Low      Close   Volume
74717  2024-09-19  76.699997  76.940002  76.089996  76.139999   205766
74718  2024-09-18  75.540001  76.690002  74.309998  75.360001  1692300
74719  2024-09-17  75.720001  75.860001  75.160004  75.529999  1316000
74720  2024-09-16  74.660004  75.

,Date,Open,High,Low,Close,Volume


In [ ]:
import talib
candle_names = talib.get_function_groups()['Pattern Recognition']
removed = ['CDLCOUNTERATTACK', 'CDLLONGLINE', 'CDLSHORTLINE',
           'CDLSTALLEDPATTERN', 'CDLKICKINGBYLENGTH']
candle_names = [name for name in candle_names if name not in removed]

In [ ]:
', '.join(candle_names)

'CDL2CROWS, CDL3BLACKCROWS, CDL3INSIDE, CDL3LINESTRIKE, CDL3OUTSIDE, CDL3STARSINSOUTH, CDL3WHITESOLDIERS, CDLABANDONEDBABY, CDLADVANCEBLOCK, CDLBELTHOLD, CDLBREAKAWAY, CDLCLOSINGMARUBOZU, CDLCONCEALBABYSWALL, CDLDARKCLOUDCOVER, CDLDOJI, CDLDOJISTAR, CDLDRAGONFLYDOJI, CDLENGULFING, CDLEVENINGDOJISTAR, CDLEVENINGSTAR, CDLGAPSIDESIDEWHITE, CDLGRAVESTONEDOJI, CDLHAMMER, CDLHANGINGMAN, CDLHARAMI, CDLHARAMICROSS, CDLHIGHWAVE, CDLHIKKAKE, CDLHIKKAKEMOD, CDLHOMINGPIGEON, CDLIDENTICAL3CROWS, CDLINNECK, CDLINVERTEDHAMMER, CDLKICKING, CDLLADDERBOTTOM, CDLLONGLEGGEDDOJI, CDLMARUBOZU, CDLMATCHINGLOW, CDLMATHOLD, CDLMORNINGDOJISTAR, CDLMORNINGSTAR, CDLONNECK, CDLPIERCING, CDLRICKSHAWMAN, CDLRISEFALL3METHODS, CDLSEPARATINGLINES, CDLSHOOTINGSTAR, CDLSPINNINGTOP, CDLSTICKSANDWICH, CDLTAKURI, CDLTASUKIGAP, CDLTHRUSTING, CDLTRISTAR, CDLUNIQUE3RIVER, CDLUPSIDEGAP2CROWS, CDLXSIDEGAP3METHODS'

In [ ]:
candle_rankings = {
        "CDL3LINESTRIKE_Bull": 1,
        "CDL3LINESTRIKE_Bear": 2,
        "CDL3BLACKCROWS_Bull": 3,
        "CDL3BLACKCROWS_Bear": 3,
        "CDLEVENINGSTAR_Bull": 4,
        "CDLEVENINGSTAR_Bear": 4,
        "CDLTASUKIGAP_Bull": 5,
        "CDLTASUKIGAP_Bear": 5,
        "CDLINVERTEDHAMMER_Bull": 6,
        "CDLINVERTEDHAMMER_Bear": 6,
        "CDLMATCHINGLOW_Bull": 7,
        "CDLMATCHINGLOW_Bear": 7,
        "CDLABANDONEDBABY_Bull": 8,
        "CDLABANDONEDBABY_Bear": 8,
        "CDLBREAKAWAY_Bull": 10,
        "CDLBREAKAWAY_Bear": 10,
        "CDLMORNINGSTAR_Bull": 12,
        "CDLMORNINGSTAR_Bear": 12,
        "CDLPIERCING_Bull": 13,
        "CDLPIERCING_Bear": 13,
        "CDLSTICKSANDWICH_Bull": 14,
        "CDLSTICKSANDWICH_Bear": 14,
        "CDLTHRUSTING_Bull": 15,
        "CDLTHRUSTING_Bear": 15,
        "CDLINNECK_Bull": 17,
        "CDLINNECK_Bear": 17,
        "CDL3INSIDE_Bull": 20,
        "CDL3INSIDE_Bear": 56,
        "CDLHOMINGPIGEON_Bull": 21,
        "CDLHOMINGPIGEON_Bear": 21,
        "CDLDARKCLOUDCOVER_Bull": 22,
        "CDLDARKCLOUDCOVER_Bear": 22,
        "CDLIDENTICAL3CROWS_Bull": 24,
        "CDLIDENTICAL3CROWS_Bear": 24,
        "CDLMORNINGDOJISTAR_Bull": 25,
        "CDLMORNINGDOJISTAR_Bear": 25,
        "CDLXSIDEGAP3METHODS_Bull": 27,
        "CDLXSIDEGAP3METHODS_Bear": 26,
        "CDLTRISTAR_Bull": 28,
        "CDLTRISTAR_Bear": 76,
        "CDLGAPSIDESIDEWHITE_Bull": 46,
        "CDLGAPSIDESIDEWHITE_Bear": 29,
        "CDLEVENINGDOJISTAR_Bull": 30,
        "CDLEVENINGDOJISTAR_Bear": 30,
        "CDL3WHITESOLDIERS_Bull": 32,
        "CDL3WHITESOLDIERS_Bear": 32,
        "CDLONNECK_Bull": 33,
        "CDLONNECK_Bear": 33,
        "CDL3OUTSIDE_Bull": 34,
        "CDL3OUTSIDE_Bear": 39,
        "CDLRICKSHAWMAN_Bull": 35,
        "CDLRICKSHAWMAN_Bear": 35,
        "CDLSEPARATINGLINES_Bull": 36,
        "CDLSEPARATINGLINES_Bear": 40,
        "CDLLONGLEGGEDDOJI_Bull": 37,
        "CDLLONGLEGGEDDOJI_Bear": 37,
        "CDLHARAMI_Bull": 38,
        "CDLHARAMI_Bear": 72,
        "CDLLADDERBOTTOM_Bull": 41,
        "CDLLADDERBOTTOM_Bear": 41,
        "CDLCLOSINGMARUBOZU_Bull": 70,
        "CDLCLOSINGMARUBOZU_Bear": 43,
        "CDLTAKURI_Bull": 47,
        "CDLTAKURI_Bear": 47,
        "CDLDOJISTAR_Bull": 49,
        "CDLDOJISTAR_Bear": 51,
        "CDLHARAMICROSS_Bull": 50,
        "CDLHARAMICROSS_Bear": 80,
        "CDLADVANCEBLOCK_Bull": 54,
        "CDLADVANCEBLOCK_Bear": 54,
        "CDLSHOOTINGSTAR_Bull": 55,
        "CDLSHOOTINGSTAR_Bear": 55,
        "CDLMARUBOZU_Bull": 71,
        "CDLMARUBOZU_Bear": 57,
        "CDLUNIQUE3RIVER_Bull": 60,
        "CDLUNIQUE3RIVER_Bear": 60,
        "CDL2CROWS_Bull": 61,
        "CDL2CROWS_Bear": 61,
        "CDLBELTHOLD_Bull": 62,
        "CDLBELTHOLD_Bear": 63,
        "CDLHAMMER_Bull": 65,
        "CDLHAMMER_Bear": 65,
        "CDLHIGHWAVE_Bull": 67,
        "CDLHIGHWAVE_Bear": 67,
        "CDLSPINNINGTOP_Bull": 69,
        "CDLSPINNINGTOP_Bear": 73,
        "CDLUPSIDEGAP2CROWS_Bull": 74,
        "CDLUPSIDEGAP2CROWS_Bear": 74,
        "CDLGRAVESTONEDOJI_Bull": 77,
        "CDLGRAVESTONEDOJI_Bear": 77,
        "CDLHIKKAKEMOD_Bull": 82,
        "CDLHIKKAKEMOD_Bear": 81,
        "CDLHIKKAKE_Bull": 85,
        "CDLHIKKAKE_Bear": 83,
        "CDLENGULFING_Bull": 84,
        "CDLENGULFING_Bear": 91,
        "CDLMATHOLD_Bull": 86,
        "CDLMATHOLD_Bear": 86,
        "CDLHANGINGMAN_Bull": 87,
        "CDLHANGINGMAN_Bear": 87,
        "CDLRISEFALL3METHODS_Bull": 94,
        "CDLRISEFALL3METHODS_Bear": 89,
        "CDLKICKING_Bull": 96,
        "CDLKICKING_Bear": 102,
        "CDLDRAGONFLYDOJI_Bull": 98,
        "CDLDRAGONFLYDOJI_Bear": 98,
        "CDLCONCEALBABYSWALL_Bull": 101,
        "CDLCONCEALBABYSWALL_Bear": 101,
        "CDL3STARSINSOUTH_Bull": 103,
        "CDL3STARSINSOUTH_Bear": 103,
        "CDLDOJI_Bull": 104,
        "CDLDOJI_Bear": 104
    }

In [ ]:
crypto.reset_index(inplace=True)
crypto = crypto[['Date', 'Open', 'High', 'Low', 'Close']]
crypto.columns = ['time', 'open', 'high', 'low', 'close']

In [ ]:
def cleanPx(prices, freq='1D'):
    if prices.empty:
        return prices  # Return the empty DataFrame if it's empty

    # Convert 'Date' to datetime and handle potential errors
    try:
        prices.Date = pd.to_datetime(prices.Date)
    except Exception as e:
        print(f"Error converting 'Date' column: {e}")
        return prices  # Or handle the error in a more appropriate way

    #prices = prices.iloc[prices.Date.drop_duplicates(keep='last').index]
    prices = prices[~prices.Date.duplicated(keep='last')]
    #prices.Date = pd.to_datetime(prices.Date)
    prices.set_index('Date', inplace=True)

    prices_ohlc = prices[['Open','High','Low','Close']]
    prices_vol = prices[['Volume']]
    prices_symbols = prices[['Symbol']]
    #prices_Date = prices[['Date']]
    #print(prices_symbols)

    prices_ohlc = prices_ohlc.resample(freq).agg({'Open': 'first',
                                 'High': 'max',
                                 'Low': 'min',
                                 'Close': 'last'})
    prices_vol = prices_vol.resample(freq).sum()
    prices_symbols = prices_symbols.resample(freq).last()
    #print(prices_symbols)

    prices = pd.concat([prices_ohlc, prices_vol, prices_symbols], axis=1)
    prices.index = prices.index.tz_localize('UTC').tz_convert('Asia/Singapore')

    return prices.dropna()

In [ ]:
#crypto = pd.read_csv("/content/historical_data.csv", names=['Date', 'Open', 'High', 'Low', 'Close', 'Symbol'], parse_dates=True)

for ticker in nasdaq_symbols:
    #try:
      crypto_org = pd.read_csv("/content/historical_data.csv", names=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Symbol'], parse_dates=True)
      crypto = crypto_org[crypto_org["Symbol"] == ticker]

      if not crypto.empty:
        crypto = cleanPx(crypto, '1D')
        #crypto.reset_index(inplace=True)
        #crypto.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Symbol']
        #crypto.set_index('Date', inplace=True)
        crypto_csv_df = pd.DataFrame(crypto)
        crypto_csv_df.reset_index(inplace=True)
        crypto_csv_df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Symbol']
        #crypto_csv_df.set_index('Date', inplace=True)
        print(crypto_csv_df.head())

        OUTPUT_FOLDER = '/content/'
        crypto_csv_df.to_csv(OUTPUT_FOLDER + 'crypto.csv', mode='a', index=False, header=False)


      #crypto
      #print(crypto.head())


    #except Exception:
      #continue

Streaming output truncated to the last 5000 lines.
2 2024-08-27 08:00:00+08:00  5.17  5.530  5.00   5.38   366100   RITR
3 2024-08-28 08:00:00+08:00  5.10  5.440  4.55   4.88   234500   RITR
4 2024-08-29 08:00:00+08:00  4.63  5.249  4.60   5.19    93000   RITR
                       Date   Open   High    Low  Close    Volume Symbol
0 2024-08-20 08:00:00+08:00  13.49  13.69  13.19  13.20  23709800   RIVN
1 2024-08-21 08:00:00+08:00  13.31  13.35  12.85  13.07  24317300   RIVN
2 2024-08-22 08:00:00+08:00  13.16  13.21  12.77  12.81  20573300   RIVN
3 2024-08-23 08:00:00+08:00  12.92  14.08  12.89  13.96  39766900   RIVN
4 2024-08-26 08:00:00+08:00  13.95  14.72  13.72  14.36  33109800   RIVN
                       Date  Open  High   Low  Close  Volume Symbol
0 2024-08-20 08:00:00+08:00  2.78  2.78  2.54  2.661    3000   RKDA
1 2024-08-21 08:00:00+08:00  2.58  2.70  2.58  2.700    2400   RKDA
2 2024-08-22 08:00:00+08:00  2.74  2.81  2.67  2.810    1600   RKDA
3 2024-08-23 08:00:00+08:00  

In [ ]:
# extract OHLC
crypto_csv = pd.read_csv("/content/crypto.csv", names=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Symbol'], parse_dates=True)
#print(crypto_csv.head())
#crypto_csv.reset_index(inplace=True)
crypto_csv = crypto_csv[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Symbol']]
crypto_csv.columns = ['time', 'open', 'high', 'low', 'close', 'Volume', 'Symbol']

op = crypto_csv['open']
hi = crypto_csv['high']
lo = crypto_csv['low']
cl = crypto_csv['close']

# create columns for each pattern
for candle in candle_names:
    # below is same as;
    # df["CDL3LINESTRIKE"] = talib.CDL3LINESTRIKE(op, hi, lo, cl)
    crypto_csv[candle] = getattr(talib, candle)(op, hi, lo, cl)

In [ ]:
from itertools import compress

crypto_csv['candlestick_pattern'] = np.nan
crypto_csv['candlestick_match_count'] = np.nan

for index, row in crypto_csv.iterrows():

    # no pattern found
    if len(row[candle_names]) - sum(row[candle_names] == 0) == 0:
        crypto_csv.loc[index,'candlestick_pattern'] = "NO_PATTERN"
        crypto_csv.loc[index, 'candlestick_match_count'] = 0
    # single pattern found
    elif len(row[candle_names]) - sum(row[candle_names] == 0) == 1:
        # bull pattern 100 or 200
        if any(row[candle_names].values > 0):
            pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bull'
            crypto_csv.loc[index, 'candlestick_pattern'] = pattern
            crypto_csv.loc[index, 'candlestick_match_count'] = 1
        # bear pattern -100 or -200
        else:
            pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bear'
            crypto_csv.loc[index, 'candlestick_pattern'] = pattern
            crypto_csv.loc[index, 'candlestick_match_count'] = 1
    # multiple patterns matched -- select best performance
    else:
        # filter out pattern names from bool list of values
        patterns = list(compress(row[candle_names].keys(), row[candle_names].values != 0))
        container = []
        for pattern in patterns:
            if row[pattern] > 0:
                container.append(pattern + '_Bull')
            else:
                container.append(pattern + '_Bear')
        rank_list = [candle_rankings[p] for p in container]
        if len(rank_list) == len(container):
            rank_index_best = rank_list.index(min(rank_list))
            crypto_csv.loc[index, 'candlestick_pattern'] = container[rank_index_best]
            crypto_csv.loc[index, 'candlestick_match_count'] = len(container)

In [ ]:
# clean up candle columns
try:
    crypto_csv.drop(candle_names, axis = 1, inplace = True)
except:
    pass

crypto_csv.loc[crypto_csv.candlestick_pattern == 'NO_PATTERN', 'candlestick_pattern'] = ''
crypto_csv.candlestick_pattern = crypto_csv.candlestick_pattern.apply(lambda x: x[3:] if isinstance(x, str) else x)


In [ ]:
OUTPUT_FOLDER = '/content/'
crypto_csv.to_csv(OUTPUT_FOLDER + 'ethusd.csv', index=False)